In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/ReinforcementLearningAidedPIDControllerTuner.git -b work20200604

# # To update the work environment
# %cd /content/drive/My Drive/ReinforcementLearningAidedPIDControllerTuner
# !git pull

%cd /content/drive/My Drive/ReinforcementLearningAidedPIDControllerTuner/works/work20200604

In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append(os.path.abspath("../../framework"))
sys.path.append(os.path.abspath("../../concrete"))
sys.path.append(os.path.abspath("../../snippet"))

In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcAgentFactory import ConcAgentFactory
from ConcEnvrionmentFactory import ConcEnvironmentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation

from framework import ClosedLoopSimulator
from framework import  ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

## S100. Misc

### SS130. define static variables

In [ ]:
concStore = ConcStore()
agentFactory = ConcAgentFactory()
environmentFactory = ConcEnvironmentFactory()

### SS120. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## S200. Prepare a simulation

### SS210. choose a build order

In [ ]:
# cur.execute('''
# Select
#     distinct buildOrderId
#     From TrainLog
#     Where buildOrderId like "casestudy1c %"
#     Order by timestamp
#     ''')

# buildOrderIdsToExport = [elm for elm, in cur.fetchall()]


# the following build order ids of which agent are used in the simulation and export thier results:
buildOrderIdsToExport = [
    "casestudy1c 7GYTONxLpVlFj2bb"
    , "casestudy1c mBn4LlEXffvdHr1Q"
    , "casestudy1c 6Ud1w3f0exYtkaoC"
    ]

### SS220. choose the iteration(timesimulation) over the build with the given build order:

select a timeSimulation(an iteration number of the specified build process) from timeSimulations ...

In [ ]:
cur.execute('''
Select
    Distinct TimeSimulation
    From TrainLog
    Order by TimeSimulation
    ''')
allDistinctTimeSimulations = [elm for elm, in cur.fetchall()]

In [ ]:
nExport = 5
timeSimulationsToExport = [allDistinctTimeSimulations[i] 
                           for i in np.linspace(1, len(allDistinctTimeSimulations)-1, nExport).astype(np.int)]

## SS230. restore the agent and the environment

In [ ]:
def restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven):
    cur.execute('''
        Select 
            trainId
            From TrainLog
            Where buildOrderId = ?
            And timeSimulation = ?
            ''', (buildOrderIdGiven, timeSimulationGiven))

    trainIdStr, = cur.fetchone()

    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(TrainId(trainIdStr))

    # restore agent
    agent = agentFactory.create(storeField.getBuildOrder())
    agent.loadFromMemento(storeField.getAgentMemento())

    # restrore environment
    environment = environmentFactory.create(storeField.getBuildOrder())
    
    # restore nSeq
    nSeq = storeField.buildOrder.getnSeq()
    
    return environment, agent, nSeq, storeField.getBuildOrder().asmWeightOnCost

## S300. Run a simulation of the given pair of (environment, agent)

In [ ]:
nSimu = 96 * 7

### SS310: run a simulation

In [ ]:
def runSimulation(environment, agent, nSeq):
    
    Y = [] # observations
    U = [] # actions

    closedLoopSimulator = ClosedLoopSimulator(environment, agent, nSeq)
    closedLoopSimulator.init()
    y = closedLoopSimulator.observationSequenceLast[-1].S_NH4 # (1, nPv)
    Y.append(y)
    for _ in range(nSimu):
        closedLoopSimulator.update()
        y = closedLoopSimulator.observationSequenceLast[-1].S_NH4 # (,)
        u = float(closedLoopSimulator.actionLast.getActionOnEnvironment()) # (,)
        Y.append(y)
        U.append(u)

    Ynumpy = np.array(Y) # (nSimu+1,)
    Unumpy = np.array(U) # (nSimu,)

    Ty = np.arange(nSimu+1) # (0, 1, ..., 9, 10)
    Tu = np.arange(nSimu) + 0.5 # (0.5, 1.5, 2.5, ..., 9.5)
    
    return Ty, Ynumpy, Tu, Unumpy

export the trace of observations and actions to a csv file:

In [ ]:
tbl = ["buildOrderId,trainIteration,t,variable,value,asmWeightOnCost".split(",")]

sql = """
Select 
    Count(*)
    From TrainLog
    Where buildOrderId = ?
    And timeSimulation = ?
"""
for buildOrderIdGiven in buildOrderIdsToExport:
    for timeSimulationGiven in timeSimulationsToExport:     
        cur.execute(sql, (buildOrderIdGiven, timeSimulationGiven,))
        count, = cur.fetchone()
        if count > 0:        
            environment, agent, nSeq, asmWeightOnCost = restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven)
            Ty, Ynumpy, Tu, Unumpy = runSimulation(environment, agent, nSeq)
            
            tbl += [*zip([buildOrderIdGiven,]*len(Ty), [timeSimulationGiven,]*len(Ty), Ty, ["S_NH4",] * len(Ty), Ynumpy, [asmWeightOnCost,]*len(Ty))]
            tbl += [*zip([buildOrderIdGiven,]*len(Tu), [timeSimulationGiven,]*len(Tu), Tu, ["DO",] * len(Ty), Unumpy, [asmWeightOnCost,]*len(Tu))]

with open("cs1c_simulation.csv", "w") as fp:
    for row in tbl:
        fp.write(",".join([str(elm) for elm in row]) + "\n")